# Statistical models in R
This notebook is covering:
1. ARIMA-models.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [4]:
%%R

hub1_name <- "the"
validation_size <- 250
test_size <- 250
window_size <- 5




In [5]:
%%R
source("arima.r")

hubs <- c("ttf", "the", "nbp")
window_sizes <- c(5, 10)

for (hub in hubs) {
  for (window_size in window_sizes) {
    arima_system(hub, validation_size, test_size, window_size, verbose = FALSE, save = TRUE)
  }
}

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: psych
Loading required package: TSA

Attaching package: ‘TSA’

The following objects are masked from ‘package:stats’:

    acf, arima

The following object is masked from ‘package:utils’:

    tar

Loading required package: forecast
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'forecast':
  method       from
  fitted.Arima TSA 
  plot.Arima   TSA 
This is forecast 8.23.0 
  Crossvalidated is a great place to get help on forecasting issues:
  http://stats.stackexchange.com/tags/forecasting.
Loading required 

In [6]:
%%R
source("arima.r")

output <- arima_system(hub1_name, validation_size, test_size, window_size, verbose = TRUE, save = TRUE)

[1] "AR: 4 I: 1 MA: 2"
[1] "the: Mean Absolute Error: 2.99701482005169"
[1] "the: Root Mean Squared Error: 3.96360344855582"


In [7]:
%%R
hub_predictions <- output$predictions$hub
hub_actuals <- output$actuals$hub
hub_forecast_comparison <- data.frame(
  Actual = hub_actuals,
  Forecasted = hub_predictions
)

In [8]:
%R -o hub_forecast_comparison
%R -o hub1_name

hub_forecast_comparison = hub_forecast_comparison

actual_trace = go.Scatter(
    x=hub_forecast_comparison.index,
    y=hub_forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=hub_forecast_comparison.index,
    y=hub_forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title=hub1_name.item()+ ': ARIMA Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()
